# Pengambilan Data

In [19]:
from tweepy import API 
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

In [20]:
import twitter_credentials
import perbaikan_singkatan
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
import string
import matplotlib.pyplot as plt
%matplotlib inline

In [21]:
# # # # TWITTER CLIENT # # # #
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)

        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        return self.twitter_client

In [22]:
# # # # TWITTER AUTHENTICATER # # # #
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        auth = OAuthHandler(twitter_credentials.CONSUMER_KEY, twitter_credentials.CONSUMER_SECRET)
        auth.set_access_token(twitter_credentials.ACCESS_TOKEN, twitter_credentials.ACCESS_TOKEN_SECRET)
        return auth

In [23]:
# # # # TWITTER STREAMER # # # #
class TwitterStreamer():
    """
    Class for streaming and processing live tweets.
    """
    def __init__(self):
        self.twitter_autenticator = TwitterAuthenticator()    

    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = TwitterListener(fetched_tweets_filename)
        auth = self.twitter_autenticator.authenticate_twitter_app() 
        stream = Stream(auth, listener)

        # This line filter Twitter Streams to capture data by the keywords: 
        stream.filter(track=hash_tag_list)

In [24]:
# # # # TWITTER STREAM LISTENER # # # #
class TwitterListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
      
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                       tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          
    def on_error(self, status):
        if status == 420:
            # Returning False on_data method in case rate limit occurs.
            return False
        print(status)

In [25]:
 class TweetAnalyzer():
    """
    Functionality for analyzing and categorizing content from tweets.
    """
         
    def clean_tweet(self, tweet):
    
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w +:\/\/ \S +)", " ", tweet).split())
    
    def analyze_sentiment(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        # analysis = analysis.translate(from_lang = 'in',to='en')

        if analysis.sentiment.polarity > 0:
            return "Positif"
        elif analysis.sentiment.polarity == 0:
            return "Netral"
        else:
            return "Negatif"
        
    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['tweets'])
        
        df['id'] = np.array([tweet.id_str for tweet in tweets])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['source'] = np.array([tweet.source for tweet in tweets])
        df['lang'] = np.array([tweet.lang for tweet in tweets])
        df['place'] = np.array([tweet.place for tweet in tweets ])
        

        return df

In [26]:
if __name__ == '__main__':

    twitter_client = TwitterClient()
    tweet_analyzer = TweetAnalyzer()
    keyword = ["cryptocurrency"]
    api = twitter_client.get_twitter_client_api()
    tweets = api.search(keyword, count=100, lang = 'en')

    df = tweet_analyzer.tweets_to_data_frame(tweets)
    df['sentiment'] = np.array([tweet_analyzer.analyze_sentiment(tweet) for tweet in df['tweets']])

    print(df)

NotTranslated: Translation API returned the input string unchanged.

# Preproses

In [16]:
def remove_punct(text):

    neg_pattern = re.compile(r'\b(' + '|'.join(perbaikan_singkatan.negations_dic.keys()) + r')\b')    
    #hapus simbol
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',text)
    text = re.sub('rt @[^\s]+',' ', text)
    text = re.sub('@[^\s]+',' ',text)
    text = re.sub(r'[^\x00-\x7F]+',' ', text)
    #Remove additional white spaces
    text = re.sub('[\s]+', ' ', text)
    #Replace #word with word
    neg_handled = neg_pattern.sub(lambda x: perbaikan_singkatan.negations_dic[x.group()], text)
    text = re.sub(r'#([^\s]+)', r'\1', neg_handled)
    #trim
    text = text.strip('\'"')
    text  = "".join([char for char in text if char not in string.punctuation])
    #text = re.sub('[0-9]+', '', text)

    return text

df['tweet_bersih'] = df['tweets'].apply(lambda x: remove_punct(x.lower()))

In [17]:
#--- REMOVE DUPLICATES
df = df.drop_duplicates(['tweet_bersih'])
df

,tweets,id,date,source,lang,place,sentiment,tweet_bersih
0,Listen to Neil Bergquist of Coinme (@Coinme) a...,1235686678888497153,2020-03-05 22:00:36,Hootsuite Inc.,en,None,Netral,listen to neil bergquist of coinme and jim ga...
1,RT @IrishGirlCrypt1: 🚨1500 #Tron #Crypto #Give...,1235686664779051008,2020-03-05 22:00:33,Twitter for Android,en,None,Positif,1500 tron crypto giveaway three winners get 5...
2,Earning #cryptocurrency for selling my stuff o...,1235686646563000320,2020-03-05 22:00:29,Listia,en,None,Netral,earning cryptocurrency for selling my stuff on...
3,RT @smart_stats: Market Density:\n\nActive cry...,1235686635666157568,2020-03-05 22:00:26,Twitter Web App,en,None,Negatif,market density active cryptocurrencies 3472 6...
4,RT @AdeldMeyer: Wow big wins for #crypto two d...,1235686635322236930,2020-03-05 22:00:26,Twitter for Android,en,None,Positif,wow big wins for crypto two days in a row fir...
...,...,...,...,...,...,...,...,...
89,RT @1CrypticPoet: 🔥 Bitcoin IRA Reveals Data H...,1235684802705133570,2020-03-05 21:53:09,Twitter Web App,en,None,Positif,bitcoin ira reveals data highlighting strong ...
90,RT @crypto_checkout: Our February Community Co...,1235684800503128065,2020-03-05 21:53:09,Twitter for Android,en,None,Positif,kami february community coin of the month axe...
92,RT @DAPS_2020: $DAPS has cutting edge #private...,1235684743401721857,2020-03-05 21:52:55,Twitter for Android,en,None,Positif,daps has cutting edge private blockchain team...
93,RT @campuscodi: Malicious Chrome extension cau...,1235684741992648704,2020-03-05 21:52:55,Twitter Web App,en,None,Netral,malicious chrome extension caught stealing le...


In [18]:
df.to_excel("Data Tanggal 6 pagi (cryptocurrency).xlsx")
df.to_csv("Data Tanggal 6 pagi (cryptocurrency).csv")

# Visualisasi Data

In [ ]:
df = pd.read_excel('myindihome.xlsx',header=0,encoding = 'unicode_escape')
#df = df.drop(columns = "Unnamed: 0")
df.count

In [ ]:
df = df.drop_duplicates(['tweet_bersih'])
df = df.set_index("source")
df = df.drop("Sociomedio Pro Telkom", axis=0)

In [ ]:
df_positif = df[df['sentiment'] == 'Positif']
df_negatif = df[df['sentiment'] == 'Negatif']
df_netral = df[df['sentiment'] == 'Netral']
df_negatif

In [ ]:
sentimen_count = df['sentiment'].value_counts()
sentimen_count

In [ ]:
objects = sentimen_count.index
y_pos = np.arange(len(objects))
performance = sentimen_count
 
plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Jumlah dalam tweets')
plt.title('Sentiment Topik Wifi.id')
 
plt.show()
plt.show()

In [ ]:
labels = sentimen_count.index
sizes = sentimen_count
colors = ['steelblue', 'red', 'gray']
explode = (0.2, 0.1, 0.1)  # explode 1st slice
 
# Plot
plt.pie(sizes, explode=explode, labels=labels, colors=colors,
        autopct='%1.1f%%', shadow=True, startangle=140)
 
plt.axis('equal')
plt.show()

In [ ]:
from wordcloud import WordCloud
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words()

In [ ]:
words_positif = ' '.join(df_positif['tweet_bersih'])
words_negatif = ' '.join(df_negatif['tweet_bersih'])
words_netral = ' '.join(df_netral['tweet_bersih'])

In [ ]:
wordcloud = WordCloud(stopwords = stopwords, background_color='white', 
                      height = 1000, width = 2000).generate(words_positif)

In [ ]:
plt.imshow(wordcloud, interpolation = "bilinear")
plt.axis('off')
plt.show()

In [ ]:
# Save the image
wordcloud.to_file("wordcloud_positif.png")

In [ ]:
wordcloud = WordCloud(stopwords = stopwords, background_color='white', 
                      height = 1000, width = 2000).generate(words_negatif)

In [ ]:
plt.imshow(wordcloud, interpolation = "bilinear")
plt.axis('off')
plt.show()

In [ ]:
# Save the image
wordcloud.to_file("wordcloud_negatif1.png")

In [ ]:
wordcloud = WordCloud(stopwords = stopwords, background_color='white', 
                      height = 1000, width = 2000).generate(words_netral)

In [ ]:
plt.imshow(wordcloud, interpolation = "bilinear")
plt.axis('off')
plt.show()

In [ ]:
# Save the image
wordcloud.to_file("wordcloud_netral.png")